In [1]:
from urllib2 import urlopen
import time
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

from bs4 import BeautifulSoup
import re
import json
import numpy as np

In [18]:
def startWebDriver():
    global driver
    options = Options()
    options.add_argument("--disable-extensions")
    driver = webdriver.Chrome(executable_path = '/home/abhishek/Downloads/chromedriver_linux64/chromedriver',options=options)
    
def getJSONObject(Link):
    HTML = urlopen(Link)
    soup = BeautifulSoup(HTML,'lxml')
    
    
        
    text = text[6:9]
    
    JSONObject = {}
    #empty_JSONObject = {}
    
    bad_report_flag = 0
    field_count = 0
    
    for item in text:
        m = re.search('(.*): (.*)',item)
        
        # @ - start of sequence tag
        # $ - end of sequence tag
        if m and m.group(1) in ['Impression','Findings','Indication']:
            field_count = field_count + 1
            JSONObject[m.group(1)] = m.group(2)
            print(m.group(1) + " : " + m.group(2)) 
        else:
            bad_report_flag = 1
            break;
    
    if field_count == 3 and bad_report_flag == 0:
        return JSONObject 
    else:
        return None

In [ ]:
from selenium.common.exceptions import NoSuchElementException

startWebDriver()

with open('reports_4.jsonl', 'a') as file:
    
    for j in range(2,9):
        
        main_link = 'https://openi.nlm.nih.gov/gridquery?it=xg&at=rr&m=3' + str(j) + '01&n=3' + str(j+1) + '00'
        driver.get(main_link)
        time.sleep(10)
        first = 1
    
        for i in range(1,101):
            
            print(i)
            #get current url
            if first != 1:
                driver.get(main_link)
                time.sleep(7)    # net is slow ?
        
            first = 0
        
            link_str = '//*[@id="grid"]/a[' + str(i) + ']/img'
            print(link_str)
        
            JSONObject = {}
        
            try:
                element = driver.find_element_by_xpath(link_str).click()
                time.sleep(5)
            
                findings = driver.find_element_by_xpath('//*[@id="content"]/div[1]/div/div[2]/div[1]/div[2]/p[3]').text
                findings = findings[10:len(findings)-1].encode('ascii')
        
                impression = driver.find_element_by_xpath('//*[@id="content"]/div[1]/div/div[2]/div[1]/div[2]/p[4]').text
                impression = impression[12:len(impression)-1].encode('ascii')
        
            except NoSuchElementException:
                continue
        
            JSONObject['Findings'] = findings
            JSONObject['Impression'] = impression
        
            print('naice')
        
            if(JSONObject):
                json.dump(JSONObject , file, ensure_ascii=False)

1
//*[@id="grid"]/a[1]/img
naice
2
//*[@id="grid"]/a[2]/img
naice
3
//*[@id="grid"]/a[3]/img
naice
4
//*[@id="grid"]/a[4]/img
naice
5
//*[@id="grid"]/a[5]/img
6
//*[@id="grid"]/a[6]/img
naice
7
//*[@id="grid"]/a[7]/img
naice
8
//*[@id="grid"]/a[8]/img
naice
9
//*[@id="grid"]/a[9]/img
naice
10
//*[@id="grid"]/a[10]/img
naice
11
//*[@id="grid"]/a[11]/img
12
//*[@id="grid"]/a[12]/img
naice
13
